In [14]:
# !unzip ../input/wutong/数智城市（湖北赛区）/WTB_HB_toUserA.zip -d ../output/WTB_HB_A/

Archive:  ../input/wutong/数智城市（湖北赛区）/WTB_HB_toUserA.zip
   creating: ../output/WTB_HB_A/toUserA/
  inflating: ../output/WTB_HB_A/toUserA/testa_nolabel.csv  
  inflating: ../output/WTB_HB_A/toUserA/train.csv  
  inflating: ../output/WTB_HB_A/toUserA/submita.csv  


In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import time
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score as auc
from sklearn.model_selection import StratifiedKFold

In [2]:
path = '../output/WTB_HB_A/toUserA/'
LABEL = 'LABEL'

In [3]:
raw_df_train = pd.read_csv(path+'train.csv')
raw_df_test = pd.read_csv(path+'testa_nolabel.csv')

In [4]:
df_all = pd.concat([raw_df_train, raw_df_test])

In [5]:
for feat in df_all.columns[:-1]:
    if df_all[feat].dtype == object:
        df_all[feat] = df_all[feat].astype('str')
        print(feat)
        le = LabelEncoder()
        df_all[feat] = le.fit_transform(df_all[feat])

CITY_ID
COUNTY_NAME
IMEI_SCENE_NAME
ADD_MYXZ_SCENE_NAME
ADD_MYXZ_TIME
RM_MYXZ_SCENE_NAME
RM_MYXZ_TIME


In [6]:
df_train = df_all[df_all[LABEL].notna()]
df_test = df_all[df_all[LABEL].isna()]

In [7]:
feats = [f for f in df_train.columns if f not in [LABEL, 'USER_ID']]

In [8]:
params = {

    'learning_rate': 0.05,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'verbose': -1,
    'seed': 2222,
    'n_jobs': 4,
}

fold_num = 5
seeds = [2222]
oof = np.zeros(len(df_train))
importance = 0
pred_y = pd.DataFrame()
for seed in seeds:
    kf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
    for fold, (train_idx, val_idx) in enumerate(kf.split(df_train[feats], df_train[LABEL])):
        print('-----------', fold)
        train = lgb.Dataset(df_train.loc[train_idx, feats],
                            df_train.loc[train_idx, LABEL])
        val = lgb.Dataset(df_train.loc[val_idx, feats],
                          df_train.loc[val_idx, LABEL])
        model = lgb.train(params, train, valid_sets=val, num_boost_round=10000,
                          early_stopping_rounds=100, verbose_eval=200)
        
        oof[val_idx] += model.predict(df_train.loc[val_idx, feats]) / len(seeds)
        pred_y['fold_%d_seed_%d' % (fold, seed)] = model.predict(df_test[feats])
        importance += model.feature_importance(importance_type='gain') / fold_num
print('AUC... ',auc(df_train[LABEL], oof))

----------- 0
Training until validation scores don't improve for 100 rounds


/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[200]	valid_0's auc: 0.922882
Early stopping, best iteration is:
[216]	valid_0's auc: 0.925343
----------- 1
Training until validation scores don't improve for 100 rounds
[200]	valid_0's auc: 0.90998
Early stopping, best iteration is:
[196]	valid_0's auc: 0.910825
----------- 2
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[56]	valid_0's auc: 0.912228
----------- 3
Training until validation scores don't improve for 100 rounds
[200]	valid_0's auc: 0.929279
Early stopping, best iteration is:
[216]	valid_0's auc: 0.930649
----------- 4
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[90]	valid_0's auc: 0.941594
AUC...  0.9072090527451737


In [9]:
feats_importance = pd.DataFrame()
feats_importance['name'] = feats
feats_importance['importance'] = importance
print(feats_importance.sort_values('importance', ascending=False)[:30])

                        name    importance
14            OPP_REGION_NUM  12124.609617
46             ADD_MYXZ_TIME   5624.063046
2              ONNET_DAY_NUM   3447.875312
3                CALLING_NUM   1997.930123
22         PHONE_NO_IMEI_NUM   1890.482660
10           GPRS_TOTAL_FLOW   1420.327210
1                COUNTY_NAME   1317.132522
18     CALLED_DIVIDE_CALLING   1296.898766
4                 CALLED_NUM   1032.240661
12           CALLED_USER_NUM    953.671941
15      FIRST_CALL_AREA_CODE    927.068591
50             USER_CRDT_LVL    770.637413
41           IMEI_SCENE_NAME    721.105718
0                    CITY_ID    655.961363
48              RM_MYXZ_TIME    618.567001
17                DISPERSION    601.490886
42             IS_BLACK_CERT    597.123131
11          CALLING_USER_NUM    533.314123
13                LAC_CI_NUM    528.840554
20         CERT_PHONE_NO_NUM    442.010233
9        VPMN_INNER_CALL_NUM    404.136497
8     ROAM_PROVIN_CALLED_NUM    318.581171
49         

In [10]:
# pred_y = pred_y #.map(lambda x: 1 if x > thre else 0)
df_test[LABEL] = pred_y.mean(axis=1)
tmp = df_test.sort_values(by=LABEL, ascending=False)

submit = tmp[:255]

submit[['USER_ID']].to_csv('../output/WTB_HB_A/ansA/lgb_baseline.csv', index=False)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [23]:
!castlecli --third local --token xxxxx(自己的token) --source ../output/WTB_HB_A/ansA/lgb_baseline.csv



     ▄████▄  ▄▄▄       ██████▄▄▄█████▓██▓   ▓█████ 
    ▒██▀ ▀█ ▒████▄   ▒██    ▒▓  ██▒ ▓▓██▒   ▓█   ▀ 
    ▒▓█    ▄▒██  ▀█▄ ░ ▓██▄  ▒ ▓██░ ▒▒██░   ▒███   
    ▒▓▓▄ ▄██░██▄▄▄▄██  ▒   ██░ ▓██▓ ░▒██░   ▒▓█  ▄ 
    ▒ ▓███▀ ░▓█   ▓██▒██████▒▒ ▒██▒ ░░██████░▒████▒
    ░ ░▒ ▒  ░▒▒   ▓▒█▒ ▒▓▒ ▒ ░ ▒ ░░  ░ ▒░▓  ░░ ▒░ ░
      ░  ▒    ▒   ▒▒ ░ ░▒  ░ ░   ░   ░ ░ ▒  ░░ ░  ░
    ░         ░   ▒  ░  ░  ░   ░       ░ ░     ░   
    ░ ░           ░  ░     ░             ░  ░  ░  ░
    ░                                              

    
〔 版本状态 〕 v0.14 
〔 校验权限 〕 有效 
〔 竞赛信息 〕 第二届中国移动“梧桐杯”大数据应用创新大赛 - 数智城市（湖北赛区） 
〔 用户信息 〕 嘴爷 
〔 提交备注 〕  
〔 文件分段 〕 [1]段 
〔 上传进度 〕 [##################################################]100%        1/1
〔 资源标识 〕 0000018004936C8B85D31E11B1D18D32 
〔 提交状态 〕 成功 

